# Libraries

In [31]:
import pickle
import gzip
import numpy as np
import random

# Creating a Network
<p>Inicialmente, criamos a seguinte classe para representar uma rede neural. Ela é inicializada a partir de um array contendo quantos neurônios serão usados para cada camada - por exemplo, uma rede com 2 neurônios na primeira camada, 3 na camada escondida e 1 na camada final, seria representada por Network([2,3,1]).\n</p>
<p>Sobre a inicialização das bias e dos pesos, a primeira camada é considerada como input e por isso não sofre alterações de bias. Todos os pesos são inicializados aleatoriamente, usando o randn para distribuições com média 0 e desvio padrão igual a 1.</p>

O método feedfoward retorna o resultado da rede caso receba um ndarry Numpy a como entrada, usando a função sigmoid definida mais abaixo.

Para aprender, a rede usará um gradient descent estocástico - que é um método interativo, uma otimização que torna o processo incremental, usando amostras dos dados a cada passo. Os dados de treino recebido é uma lista de tuplas (x,y) com os dados de treino e suas verdadeiras saídas. Eta é o learning rate utilizado. Test_data é opcional e vai ser usado para testar o progresso da rede depois de cada epoch de treinamento e printa o resultado parcial - porém vai deixar todo o processo mais lento. Número de epochs será quantas vezes os pesos da rede neural irá mudar. Mini_batch_size será o tamanho de cada amostra usada no processo do SGD.

### Usando o update_mini_batch
É aqui que os pesos e bias da rede serão atualizados, utilizando backpropagation para calcular de forma mais rápida o gradiente da função custo - computando isso para cada exemplo de treino no mini_batch, que é uma parte aleatória do nosso conjunto de treinamento.



In [32]:
class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y,1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for x, y in zip(sizes[:-1],sizes[1:])]
        
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))
   
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
        
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

## Building the sigmoid function
Ao invés de utilizarmos a forma que utiliza somatório para calcular o Sigmoid(uma versão suavizada da função step - que retorna sempre 0 e 1), definida como a expressão:
<p align="middle">1/(1+exp(−∑jwjxj−b))</p>
Utilizaremos a versão vetorizada da expressão, definida como:
<p align="middle">a' = σ(w*a+b)</p>

Onde a é o vetor de ativação da primeira camada a ser processada, w é o vetor de pesos e b é o vetor de bias. Em seguida, é aplicada a função sigmoid, retornando a equação:

<p align="middle">1/(1+exp(w*a+b)) </p>

In [33]:
def sigmoid(z):
    return 1/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

## Loading Data from MNIST
Utilizaremos o load data feito pelo tutorial utilizado como referência

In [34]:

def load_data():
    with gzip.open('../data/mnist.pkl.gz','rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        training_data, validation_data, test_data = u.load()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [35]:
training_data, validation_data, test_data = load_data_wrapper()


In [36]:
net = Network([784,30,10])
net.SGD(training_data,30,10,3.0,test_data = test_data)

Epoch 0: 9040 / 10000
Epoch 1: 9201 / 10000
Epoch 2: 9274 / 10000
Epoch 3: 9340 / 10000
Epoch 4: 9346 / 10000
Epoch 5: 9402 / 10000
Epoch 6: 9402 / 10000
Epoch 7: 9412 / 10000
Epoch 8: 9415 / 10000
Epoch 9: 9431 / 10000
Epoch 10: 9433 / 10000
Epoch 11: 9445 / 10000
Epoch 12: 9461 / 10000
Epoch 13: 9464 / 10000
Epoch 14: 9426 / 10000
Epoch 15: 9445 / 10000
Epoch 16: 9469 / 10000
Epoch 17: 9492 / 10000
Epoch 18: 9486 / 10000
Epoch 19: 9470 / 10000
Epoch 20: 9506 / 10000
Epoch 21: 9484 / 10000
Epoch 22: 9482 / 10000
Epoch 23: 9499 / 10000
Epoch 24: 9479 / 10000
Epoch 25: 9503 / 10000
Epoch 26: 9484 / 10000
Epoch 27: 9484 / 10000
Epoch 28: 9477 / 10000
Epoch 29: 9477 / 10000


In [38]:
net2 = Network([784, 100, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9494 / 10000
Epoch 1: 9507 / 10000
Epoch 2: 9495 / 10000
Epoch 3: 9458 / 10000
Epoch 4: 9508 / 10000
Epoch 5: 9493 / 10000
Epoch 6: 9484 / 10000
Epoch 7: 9480 / 10000
Epoch 8: 9496 / 10000
Epoch 9: 9507 / 10000
Epoch 10: 9496 / 10000
Epoch 11: 9494 / 10000
Epoch 12: 9459 / 10000
Epoch 13: 9509 / 10000
Epoch 14: 9494 / 10000
Epoch 15: 9489 / 10000
Epoch 16: 9488 / 10000
Epoch 17: 9499 / 10000
Epoch 18: 9496 / 10000
Epoch 19: 9458 / 10000
Epoch 20: 9503 / 10000
Epoch 21: 9499 / 10000
Epoch 22: 9497 / 10000
Epoch 23: 9506 / 10000
Epoch 24: 9496 / 10000
Epoch 25: 9495 / 10000
Epoch 26: 9483 / 10000
Epoch 27: 9510 / 10000
Epoch 28: 9485 / 10000
Epoch 29: 9501 / 10000
